In [ ]:
import nbimporter
import math
import numpy as np
import sys
if('..' not in sys.path):
    sys.path.insert(0,'..')
from Maths.Action import Action
from Maths.State import State
from Maths.Cord import Cord
from Maze.Maze import Maze
from Agents.Prey import Prey
import random

class Spider(Predator):
    sid=0
    def __init__(self, maze):
        super().__init__("Spider",Spider.sid)  
        Spider.sid+=1
        self.S=State.ENTRANCE
        self.m=maze
        #self.Pos=Cord(maze.getInitialX(),maze.getInitialY())
        #self.RandomWalk()
        
    def NextMove(self):
        direction=self.RandomWalk()        
        self.Do(direction,self.m)
        
    def RandomWalk(self):
        now=self.getPos()
        possibleDirections=self.m.WhichWayIsClear(now)
        direction=possibleDirections[random.randint(0,len(possibleDirections)-1)]
        return direction
        
    def Do(self, direction, m):
        if(direction==0):
            self.MoveNorth(m)
        if(direction==1):
            self.MoveSouth(m)
        if(direction==2):
            self.MoveEast(m)
        if(direction==3):
            self.MoveWest(m)
        if(direction==4):
            self.StayStill(m)
        newC=self.getPos()
        if(newC in m.getExit()):
            self.S=State.EXIT
        else:
            self.S=State.CLEAR
            
    def getSetView(self,maze, span):
        views=[]
        for i in range(0,maze.getHeight()):
            for j in range(0, maze.getWidth()):
                c= Cord(i,j)
                view=self.getView(c,span)
                if(view not in views):
                    views.append(view)
        return views
    
    def getView(self,c, span):
        #view=[[]for i in range(2*span+1)]
        view=[]
        for i in range(c.X-span,c.X+span+1):
            row=[]
            for j in range(c.Y-span,c.Y+span+1):
                row.append(self.m.getMazePart(i,j))
            #view[i-c.X-span]=row
            view.append(row)
        return view
        
    def getReward(self, pos, possible, oldPos, view, preyPos):
        
        if not(possible):
            return -500
        else:
            if(self.m.CheckWall(pos)):
                return -50
            if(self.m.CheckClear(pos)):
                if (pos in preyPos):
                    return 1000
                for row in view:
                    for prey in preyPos:
                        if (prey in row):
                            if((math.pow(prey.X-pos.X,2)+math.pow(prey.Y-pos.Y,2))<(math.pow(prey.X-oldPos.X,2)+math.pow(prey.Y-oldPos.Y,2))):
                                return 3
                if (oldPos.equals(pos)):
                    return -20
                else:
                    return -3